In [13]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris, load_wine, fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error

In [14]:
#Custom Vs. Scikit Learn Built Decision Tree
#Building a Custom Decision Tree with Information Gain

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {"class": unique_classes[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            return {"class": np.bincount(y).argmax()}

        best_info_gain = -float("inf")
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])

            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                info_gain = self._information_gain(
                    y, y[left_mask], y[right_mask]
                )

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        "feature_idx": feature_idx,
                        "threshold": threshold,
                        "left_mask": left_mask,
                        "right_mask": right_mask,
                    }

        if best_split is None:
            return {"class": np.bincount(y).argmax()}

        left_tree = self._build_tree(
            X[best_split["left_mask"]],
            y[best_split["left_mask"]],
            depth + 1,
        )

        right_tree = self._build_tree(
            X[best_split["right_mask"]],
            y[best_split["right_mask"]],
            depth + 1,
        )

        return {
            "feature_idx": best_split["feature_idx"],
            "threshold": best_split["threshold"],
            "left_tree": left_tree,
            "right_tree": right_tree,
        }

    def _information_gain(self, parent, left, right):
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_entropy = (
            len(left) / len(parent) * left_entropy
            + len(right) / len(parent) * right_entropy
        )

        return parent_entropy - weighted_entropy

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def predict(self, X):
        return [self._predict_single(x, self.tree) for x in X]

    def _predict_single(self, x, tree):
        if "class" in tree:
            return tree["class"]

        if x[tree["feature_idx"]] <= tree["threshold"]:
            return self._predict_single(x, tree["left_tree"])
        else:
            return self._predict_single(x, tree["right_tree"])

In [15]:
#Load and Split the IRIS Dataset

data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [16]:
#Train & Evaluate Custom Decision Tree

custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 1.0000


In [17]:
#Train & Evaluate Scikit-Learn Decision Tree

sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

y_pred_sklearn = sklearn_tree.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


In [18]:
#Accuracy Comparison

print("Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


In [19]:
#Ensemble Methods and Hyperparameter Tuning(Wine Dataset)
#Implement Classification Models:

wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)
y_pred_rf = rf.predict(X_test)

print("Decision Tree F1 Score:", f1_score(y_test, y_pred_dt, average="weighted"))
print("Random Forest F1 Score:", f1_score(y_test, y_pred_rf, average="weighted"))

Decision Tree F1 Score: 0.9439974457215836
Random Forest F1 Score: 1.0


In [20]:
#Hyperparameter Tuning (Random Forest – GridSearchCV)

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10],
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring="f1_weighted",
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best F1 Score: 0.9782952128219708


In [21]:
#Implement Regression Model
#Train a Decision Tree Regressor and a Random Forest Regressor using scikit-learn.

housing = fetch_california_housing()
X = housing.data
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt_reg = DecisionTreeRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)

dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

print("Decision Tree MSE:", mean_squared_error(y_test, dt_reg.predict(X_test)))
print("Random Forest MSE:", mean_squared_error(y_test, rf_reg.predict(X_test)))

Decision Tree MSE: 0.495235205629094
Random Forest MSE: 0.2553684927247781


In [12]:
#Identify three parameters for Random Forest Regressio and Perform hyperparameter tuning using RandomSearchCV to optimize these parameters.

param_dist = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "max_features": ["sqrt", "log2"],
}

random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best MSE:", -random_search.best_score_)

Best Parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 20}
Best MSE: 0.2458085852355155
